In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import load, dump

# load files

In [2]:
mode = "test"

In [3]:
# train of test
df = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/{}_shifted_rounded.parquet".format(mode))

In [4]:
all_cols = [c for c in list(df.columns) if c not in ["customer_ID", "S_2", "first_occurance", "time_id", "end_year_month"]]
nan_related_features = [
    "number_of_observations",
    "type_0_nan",
    "type_1_nan",
    "type_2_nan"
]
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]
num_features = [col for col in all_cols if col not in (cat_features + nan_related_features)]

# sma

In [5]:
sma_num_agg_4 = df.loc[df["time_id"] >= 4].groupby("customer_ID")[num_features].agg(np.nanmean)
sma_num_agg_4.columns = [(x + "_nanmean_4") for x in sma_num_agg_4.columns]

sma_num_agg_7 = df.loc[df["time_id"] >= 7].groupby("customer_ID")[num_features].agg(np.nanmean)
sma_num_agg_7.columns = [(x + "_nanmean_7") for x in sma_num_agg_7.columns]

sma_num_agg_10 = df.loc[df["time_id"] >= 10].groupby("customer_ID")[num_features].agg(np.nanmean)
sma_num_agg_10.columns = [(x + "_nanmean_10") for x in sma_num_agg_10.columns]

In [6]:
sma_num_agg = pd.concat([sma_num_agg_4, sma_num_agg_7, sma_num_agg_10], axis=1)

In [7]:
sma_num_agg.head()

,P_2_nanmean_4,D_39_nanmean_4,B_1_nanmean_4,B_2_nanmean_4,R_1_nanmean_4,S_3_nanmean_4,D_41_nanmean_4,B_3_nanmean_4,D_42_nanmean_4,D_43_nanmean_4,...,D_136_nanmean_10,D_137_nanmean_10,D_138_nanmean_10,D_139_nanmean_10,D_140_nanmean_10,D_141_nanmean_10,D_142_nanmean_10,D_143_nanmean_10,D_144_nanmean_10,D_145_nanmean_10
customer_ID,,,,,,,,,,,,,,,,,,,,,
-9223277493928322471,0.745556,0.000000,0.004444,0.837778,0.0,0.196667,0.0,0.006667,NaN,0.117778,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,0.0,0.006667,0.0
-9223220269070810982,0.841111,0.000000,0.017778,0.836667,0.0,0.010000,0.0,0.024444,NaN,0.081667,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,0.0,0.006667,0.0
-9223219380479694318,0.781111,0.000000,0.007778,0.875556,0.0,0.263333,0.0,0.011111,NaN,0.111250,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,0.0,0.000000,0.0
-9223202973368451495,0.730000,0.000000,0.005556,0.815556,0.0,0.163333,0.0,0.006667,NaN,0.051111,...,NaN,NaN,NaN,1.0,0.0,0.86,0.0,1.0,0.010000,2.0
-9223190037945288673,0.953333,4.333333,0.037778,1.004444,0.0,0.156667,0.0,0.008889,NaN,0.010000,...,NaN,NaN,NaN,0.0,0.0,0.00,NaN,0.0,0.006667,0.0


In [8]:
sma_num_agg.to_parquet("../input/{}_sma_agg_features.parquet".format(mode))